In [1]:
import json
import numpy as np
import pandas as pd
from numba import njit
from csc.atoms import *
from csc.dictionary import *
import matplotlib.pyplot as plt

# atom parameters
bmin = 0.005
bmax = 0.60
b_step = 0.005
b_values = np.arange(bmin, bmax, b_step)

ymin = 0.01
ymax = 0.034
y_step = 0.002
y_values = np.arange(ymin, ymax, y_step)

s_values = [1]

dictionary = ZSDictionary.from_values(b_values, y_values, s_values)
print('The dictionary has {} different atoms'.format(len(dictionary)))

The dictionary has 1428 different atoms


In [2]:
sparsity_level = 5
batch_size = 200    
filename = f'output_posErrPipeline_s{sparsity_level}_b{batch_size}.json'

In [3]:
try:
    with open(filename, 'r') as json_file:
        data = json.load(json_file)
        print("Données chargées avec succès:")
        print(data)
except FileNotFoundError:
    print(f"Le fichier {filename} n'a pas été trouvé.")
except json.JSONDecodeError:
    print(f"Le fichier {filename} n'est pas un JSON valide.")

Données chargées avec succès:
{'0': {'-322.0': 1, '-199.0': 2, '55.0': 4, '-1.0': 46, '0.0': 54, '-121.0': 1, '-96.0': 2, '1.0': 48, '-51.0': 2, '227.0': 1, '-114.0': 1, '-130.0': 2, '-8.0': 4, '-2.0': 31, '2.0': 27, '-449.0': 1, '-126.0': 1, '49.0': 2, '-236.0': 1, '26.0': 5, '-23.0': 5, '3.0': 19, '136.0': 2, '20.0': 3, '12.0': 1, '-18.0': 3, '-98.0': 4, '317.0': 1, '302.0': 1, '-157.0': 2, '101.0': 5, '5.0': 7, '-32.0': 6, '149.0': 3, '-42.0': 3, '356.0': 1, '-54.0': 4, '-87.0': 3, '-53.0': 3, '-29.0': 6, '7.0': 2, '64.0': 2, '-3.0': 18, '33.0': 5, '-365.0': 1, '-69.0': 3, '-407.0': 1, '35.0': 6, '25.0': 3, '222.0': 2, '27.0': 4, '-70.0': 2, '-111.0': 2, '-28.0': 4, '-84.0': 4, '223.0': 1, '272.0': 1, '-85.0': 3, '-4.0': 15, '-131.0': 1, '117.0': 2, '176.0': 1, '-71.0': 1, '-39.0': 5, '6.0': 6, '-77.0': 2, '39.0': 3, '-5.0': 5, '24.0': 2, '-79.0': 3, '-320.0': 1, '178.0': 1, '353.0': 1, '289.0': 2, '-24.0': 3, '-167.0': 3, '133.0': 3, '-27.0': 2, '159.0': 1, '-94.0': 1, '9.0': 5, '-

In [4]:
# Exemple d'utilisation des données
if data:  # Assurez-vous que les données ne sont pas vides
    for key, value in data.items():
        print(f"Niveau de bruit: {key}, Données: {value}")
        print(max(data[key].keys()))
        print(min(data[key].keys()))


Niveau de bruit: 0, Données: {'-322.0': 1, '-199.0': 2, '55.0': 4, '-1.0': 46, '0.0': 54, '-121.0': 1, '-96.0': 2, '1.0': 48, '-51.0': 2, '227.0': 1, '-114.0': 1, '-130.0': 2, '-8.0': 4, '-2.0': 31, '2.0': 27, '-449.0': 1, '-126.0': 1, '49.0': 2, '-236.0': 1, '26.0': 5, '-23.0': 5, '3.0': 19, '136.0': 2, '20.0': 3, '12.0': 1, '-18.0': 3, '-98.0': 4, '317.0': 1, '302.0': 1, '-157.0': 2, '101.0': 5, '5.0': 7, '-32.0': 6, '149.0': 3, '-42.0': 3, '356.0': 1, '-54.0': 4, '-87.0': 3, '-53.0': 3, '-29.0': 6, '7.0': 2, '64.0': 2, '-3.0': 18, '33.0': 5, '-365.0': 1, '-69.0': 3, '-407.0': 1, '35.0': 6, '25.0': 3, '222.0': 2, '27.0': 4, '-70.0': 2, '-111.0': 2, '-28.0': 4, '-84.0': 4, '223.0': 1, '272.0': 1, '-85.0': 3, '-4.0': 15, '-131.0': 1, '117.0': 2, '176.0': 1, '-71.0': 1, '-39.0': 5, '6.0': 6, '-77.0': 2, '39.0': 3, '-5.0': 5, '24.0': 2, '-79.0': 3, '-320.0': 1, '178.0': 1, '353.0': 1, '289.0': 2, '-24.0': 3, '-167.0': 3, '133.0': 3, '-27.0': 2, '159.0': 1, '-94.0': 1, '9.0': 5, '-19.0': 

In [5]:
plt.figure(figsize=(10, 6))

# Pre-processing
results_dict = data
# Génère des couleurs
colors = plt.cm.viridis(np.linspace(0, 1, len(results_dict)))  

# Définition des bins pour les erreurs de position
max_error = max(max(counter.keys()) for counter in results_dict.values())
min_error = min(min(counter.keys()) for counter in results_dict.values())
bins = np.arange(min_error, max_error + 50, 50)  # Création des bins de 50 unités

# Parcourir le dictionnaire de résultats et dessiner un histogramme pour chaque niveau de bruit
for (noise_level, counter), color in zip(results_dict.items(), colors):
    # Regroupement des erreurs par bins
    errors, counts = zip(*counter.items())
    summed_counts, _ = np.histogram(errors, bins=bins, weights=counts)
    
    # Normaliser pour faire un histogramme de densité
    summed_counts = summed_counts / sum(summed_counts)

    # Dessiner l'histogramme de densité
    plt.bar(bins[:-1], summed_counts, color=color, alpha=0.8, label=f'Noise {noise_level:.2f}', width=45)

plt.title('3 atoms/signal & 20 signals/batch')
plt.xlabel('Position Error')
plt.ylabel('Density')
plt.xticks(bins)  # Définir les ticks sur les bins
plt.legend(title="Noise Levels")
plt.grid(True)
plt.show()

TypeError: can only concatenate str (not "int") to str

<Figure size 1000x600 with 0 Axes>